In [ ]:
import requests
import pandas as pd
import json

api_token = 'sk-a94e722134454665b8fef72ddb349f37'

# 測試用 DataFrame
df = pd.DataFrame({
    "text": ["我們都是為你好", "今天天氣很好", "你怎麼這麼不懂事"]
})

url = "http://localhost:3000/api/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_token}"
}
results = []
scores, types, reasons = [], [], []

for row in df["text"]:
    data = {
                    "model": "mistral:latest",
                    "knowledge": ["pua_db"],
                    "messages": [
                        {"role": "system", "content": """你是一個情緒勒索分析助手。 請根據知識庫文件，對給定文字做以下三個任務，並輸出 JSON 格式：
                        1. "score": 情緒勒索的分數 1~10 1是最低分，10是勒索最嚴重的
                        2. "type": 情緒勒索的類型（例如：威脅、責怪、冷暴力、罪惡感操控、貶低、其他）
                        3. "reason": 一句話簡短說明為什麼這樣判斷

                        ⚠️ 請務必只輸出 JSON，不要多餘文字。"""},
                        {"role": "user", "content": f"請分析這段文字: {row}"}
                                ]
                            }

    res = requests.post(url, headers=headers, json=data)
    content = res.json()["choices"][0]["message"]["content"]
    print(content)

    try:
        parsed = json.loads(content)
        scores.append(parsed.get("score"))
        types.append(parsed.get("type"))
        reasons.append(parsed.get("reason"))
    except Exception:
        scores.append(None)
        types.append(None)
        reasons.append(content)  # 錯誤時存原始輸出

df["score"] = scores
df["type"] = types
df["reason"] = reasons

print(df)



 {
  "score": "7",
  "type": "貶低",
  "reason": "使用了「為」表示對方感激之意，但同時使用了「都」，讓文字中有些刺鼻，似乎指 的其他人一起受害"
}
 {
  "score": 3,
  "type": "其他",
  "reason": "此文本沒有顯示任何情緒勒索的症號，而是一個描述天氣的簡單文字。"
}
 {
  "score": "4",
  "type": "貶低",
  "reason": "使用了貼貼意的語調，但並沒有正面表達，反而對目標人士發表了一些讓人覺得他不會理解事情的意見"
}
       text score type                                          reason
0   我們都是為你好     7   貶低  使用了「為」表示對方感激之意，但同時使用了「都」，讓文字中有些刺鼻，似乎指 的其他人一起受害
1    今天天氣很好     3   其他                 此文本沒有顯示任何情緒勒索的症號，而是一個描述天氣的簡單文字。
2  你怎麼這麼不懂事     4   貶低   使用了貼貼意的語調，但並沒有正面表達，反而對目標人士發表了一些讓人覺得他不會理解事情的意見


In [22]:
df

,text,score,type,reason
0,我們都是為你好,7,貶低,使用了「為」表示對方感激之意，但同時使用了「都」，讓文字中有些刺鼻，似乎指 的其他人一起受害
1,今天天氣很好,3,其他,此文本沒有顯示任何情緒勒索的症號，而是一個描述天氣的簡單文字。
2,你怎麼這麼不懂事,4,貶低,使用了貼貼意的語調，但並沒有正面表達，反而對目標人士發表了一些讓人覺得他不會理解事情的意見
